# Fitting <a class="label-experiment" href="https://docs.easydiffraction.org/lib/glossary/#experiment-type-labels">pd-neut-tof</a> Si-SEPD

This example shows how to refine the crystal structure parameters of Si from neutron
diffraction data in a time-of-flight experiment performed on SEPD diffractometer at Argonne.

In [ ]:
# Check if the easydiffraction library is installed
# If not, install it including the 'charts' extras
# This is done when running in a Jupyter notebook
import builtins
import importlib.util

if hasattr(builtins, "__IPYTHON__"):
    if importlib.util.find_spec('easydiffraction') is None:
        print('Installing the easydiffraction library...')
        !pip install 'easydiffraction[charts]'

## Import EasyDiffraction

In [ ]:
import easydiffraction as ed

## Create a job

Create a job — the main object to store all the information

In [ ]:
job = ed.Job(type='tof')
print(job.type)

## Define a model

Create a phase object

In [ ]:
phase = ed.Phase(name='si')

Set space group

In [ ]:
phase.space_group.name_hm_alt = 'F d -3 m'
print(phase.space_group.name_hm_alt)
print(phase.space_group.it_coordinate_system_code)

Set cell parameters

In [ ]:
phase.cell.length_a = 5.43146

Add atoms

In [ ]:
phase.atom_sites.append(label='Si',
                        type_symbol='Si',
                        fract_x=0.125,
                        fract_y=0.125,
                        fract_z=0.125,
                        occupancy=1,
                        b_iso_or_equiv=0.529)

Add phase to the job object

In [ ]:
job.add_phase(phase=phase)
print(job.phases)

Show phase info in CIF format

In [ ]:
phase = job.phases['si']
print(phase.cif)

Display the crystal structure of a given model

In [ ]:
job.show_crystal_structure(id='si')

## Define an experiment

Download the data file from the EasyDiffraction repository on GitHub

In [ ]:
ed.download_from_repository('sepd.xye', destination='data')

Show content of the downloaded file (first 5 lines)

In [ ]:
with open('data/sepd.xye') as f:
    print(''.join(f.readlines()[:5]))

Load experimentally measured data from a file in XYE format

In [ ]:
job.add_experiment_from_file('data/sepd.xye')

Display the experimentally measured data

In [ ]:
job.show_experiment_chart(show_legend=False)

Define a point background

In [ ]:
background_points = [(x, 200) for x in range(0, 35000, 5000)]
print(background_points)

In [ ]:
job.set_background(background_points)

Display the experiment chart after setting the background

In [ ]:
job.show_experiment_chart()

## Perform an analysis

Display the analysis chart before setting initial parameter values

In [ ]:
job.show_analysis_chart()

Create aliases for the two types of experimental parameters

In [ ]:
pattern_params = job.pattern
experiment_params = job.parameters

Change the default value of some instrumental parameters and display the analysis chart again

In [ ]:
experiment_params.dtt1 = 7476.91
experiment_params.dtt2 = -1.54
experiment_params.ttheta_bank = 144.845
job.show_analysis_chart()

Change the scale and display the analysis chart again

In [ ]:
phase.scale = 10
job.show_analysis_chart()

Change the default values of the peak profile related parameters and display the analysis chart again

In [ ]:
experiment_params.alpha0 = 0.024
experiment_params.alpha1 = 0.204
experiment_params.beta0 = 0.038
experiment_params.beta1 = 0.011
experiment_params.sigma0 = 0.0
experiment_params.sigma1 = 0.0
experiment_params.sigma2 = 0.0
job.show_analysis_chart()

Select parameters to be refined in the first round

In [ ]:
phase.scale.free = True
pattern_params.zero_shift.free = True

Print parameters to be refined (free parameters) before fitting

In [ ]:
job.show_free_parameters()

Start Least-Squares minimization to refine the selected parameters

In [ ]:
job.fit()

Print the refined parameters after fitting

In [ ]:
job.show_free_parameters()

Display the analysis chart after the first fitting

In [ ]:
job.show_analysis_chart()

Allow background parameters to be refined

In [ ]:
for background_point in pattern_params.backgrounds[0]:
    background_point.y.free = True

Print free parameters before the second fitting

In [ ]:
job.show_free_parameters()

Start second round of minimization

In [ ]:
job.fit()

Print free parameters after the second fitting

In [ ]:
job.show_free_parameters()

Display the analysis chart after the second fitting

In [ ]:
job.show_analysis_chart()

Select more parameters to be refined in the third round

In [ ]:
experiment_params.sigma0.free = True
experiment_params.sigma1.free = True
experiment_params.sigma2.free = True

Disable refinement of background parameters

In [ ]:
for background_point in pattern_params.backgrounds[0]:
    background_point.y.free = False

Print free parameters before the third fitting

In [ ]:
job.show_free_parameters()

Start third round of minimization

In [ ]:
job.fit()

Print free parameters after the third fitting

In [ ]:
job.show_free_parameters()

Display the analysis chart after the third fitting

In [ ]:
job.show_analysis_chart()